In [ ]:
import holoviews as hv; hv.extension("bokeh", logo=False)
import panel as pn; pn.extension()

import numpy as np
import os

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
plt.rcParams['figure.figsize'] = [16, 8]
plt.rcParams.update({'font.size': 18})

In [ ]:
# Load dataset
H = np.loadtxt(os.path.join('..','DATA','housing.data'))
b = H[:,-1] # housing values in $1000s
A = H[:,:-1] # other factors

# Pad with ones for nonzero offset
A = np.pad(A,[(0,0),(0,1)],mode='constant',constant_values=1)


# Solve Ax=b using SVD
# Note that the book uses the Matlab-specific "regress" command
U, S, VT = np.linalg.svd(A,full_matrices=0)
x = VT.T @ np.linalg.inv(np.diag(S)) @ U.T @ b

In [ ]:
sort_ind = np.argsort(H[:,-1])

hv.Scatter(b, "Neighborhood", 'Median Home Value [$1k]', label='Housing Value').opts(width=400, show_grid=True, tools=['hover'], color='red', size=5)*\
hv.Curve(A@x, label='Regression').opts(tools=['hover'], alpha=0.6) +\
(hv.Scatter(b[sort_ind], "Neighborhood", 'Median Home Value [$1k]', label='Housing Value').opts(width=400, show_grid=True, tools=['hover'], color='red', size=5)*\
hv.Curve(A[sort_ind,:]@x, label='Regression').opts(tools=['hover'], alpha=0.6)).opts( legend_position='top_left')

In [ ]:
A_mean = np.mean(A,axis=0)
A_mean = A_mean.reshape(-1, 1)

A2 = A - np.ones((A.shape[0],1)) @ A_mean.T

for j in range(A.shape[1]-1):
    A2std = np.std(A2[:,j])
    A2[:,j] = A2[:,j]/A2std
    
A2[:,-1] = np.ones(A.shape[0])

U, S, VT = np.linalg.svd(A2,full_matrices=0)
x = VT.T @ np.linalg.inv(np.diag(S)) @ U.T @ b
x_tick = range(len(x)-1)+np.ones(len(x)-1)

In [ ]:
hv.Bars((x_tick,x[:-1]), 'Attribute', 'Significance').opts(width=500)